# Tessellation/Lattice Approach for Large N

Implementing the tessellation approach from egortrushin kernel for large N values (N >= 58).

Key insight: Use 2 base trees in a grid pattern with translation vectors.
- Blue tree: angle ~0° (pointing up)
- Pink tree: angle ~180° (pointing down)
- Grid size: nt = [rows, cols] where rows * cols >= N
- Translation vectors: (dx, dy) optimized via SA

In [ ]:
import datetime
import copy
import math
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import json
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25
scale_factor = Decimal("1e15")

print("Libraries loaded")

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x="0", center_y="0", angle="0"):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x).replace('s', ''))
        self.center_y = Decimal(str(center_y).replace('s', ''))
        self.angle = Decimal(str(angle).replace('s', ''))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (Decimal("0.0") * scale_factor, tip_y * scale_factor),
                (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
                (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
                (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
                (
                    -(trunk_w / Decimal("2")) * scale_factor,
                    trunk_bottom_y * scale_factor,
                ),
                (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
                (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
                (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

    def clone(self) -> "ChristmasTree":
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print("ChristmasTree class defined")

In [ ]:
def calculate_score(trees):
    """Calculate score for a list of trees"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    score = max(max_x - min_x, max_y - min_y) ** 2 / len(trees)
    return score

def get_side_length(trees):
    """Get side length of bounding box"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def has_collision(trees):
    """Check for collisions between trees"""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

print("Helper functions defined")

In [ ]:
class SimulatedAnnealing:
    """SA optimizer for tessellation approach"""
    def __init__(
        self,
        trees,
        nt,
        Tmax,
        Tmin,
        nsteps,
        nsteps_per_T,
        cooling,
        alpha,
        n,
        position_delta,
        angle_delta,
        delta1,
        random_state,
        log_freq,
    ):
        self.trees = trees
        self.nt = nt
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        self.nsteps_per_T = nsteps_per_T
        self.cooling = cooling
        self.alpha = alpha
        self.n = n
        self.position_delta = position_delta
        self.angle_delta = angle_delta
        self.delta1 = delta1
        self.log_freq = log_freq
        random.seed(random_state)

    def perturb_tree(self, tree):
        """Perturb tree position and angle"""
        old_x, old_y, old_angle = tree.get_params()
        dx = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dy = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dangle = Decimal(str(random.uniform(-self.angle_delta, self.angle_delta)))
        new_x = old_x + dx
        new_y = old_y + dy
        new_angle = (old_angle + dangle) % 360
        tree.set_params(new_x, new_y, new_angle)
        return old_x, old_y, old_angle

    def get_length(self, current_trees):
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in current_trees])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        return max(max_x - min_x, max_y - min_y)

    def get_trees(self, delta1):
        """Generate trees using tessellation pattern"""
        trees = []
        for i in range(self.nt[0]):
            for j in range(self.nt[1]):
                if len(trees) >= self.n:
                    break
                tree_type = (i + j) % 2
                base_tree = self.trees[tree_type]
                x = base_tree.center_x + Decimal(str(i * delta1[0]))
                y = base_tree.center_y + Decimal(str(j * delta1[1]))
                trees.append(ChristmasTree(str(x), str(y), str(base_tree.angle)))
            if len(trees) >= self.n:
                break
        return trees

    def solve(self):
        T = self.Tmax
        delta1 = list(self.delta1)
        best_length = float('inf')
        best_trees = None
        best_delta1 = delta1.copy()
        
        for step in range(self.nsteps):
            for _ in range(self.nsteps_per_T):
                # Perturb base trees
                old_params = []
                for tree in self.trees:
                    old_params.append(self.perturb_tree(tree))
                
                # Perturb delta1
                old_delta1 = delta1.copy()
                delta1[0] += random.uniform(-self.position_delta, self.position_delta)
                delta1[1] += random.uniform(-self.position_delta, self.position_delta)
                
                # Generate trees and check
                current_trees = self.get_trees(delta1)
                
                if has_collision(current_trees):
                    # Revert
                    for i, tree in enumerate(self.trees):
                        tree.set_params(*old_params[i])
                    delta1 = old_delta1
                    continue
                
                current_length = self.get_length(current_trees)
                
                if current_length < best_length:
                    best_length = current_length
                    best_trees = [t.clone() for t in current_trees]
                    best_delta1 = delta1.copy()
                    if step % self.log_freq == 0:
                        print(f"Step {step}: New best length = {best_length:.6f}")
                elif random.random() < math.exp(-(current_length - best_length) / T):
                    pass  # Accept worse solution
                else:
                    # Revert
                    for i, tree in enumerate(self.trees):
                        tree.set_params(*old_params[i])
                    delta1 = old_delta1
            
            # Cooling
            if self.cooling == "exponential":
                T = T * self.alpha
            elif self.cooling == "linear":
                T = T - (self.Tmax - self.Tmin) / self.nsteps
            
            T = max(T, self.Tmin)
        
        if best_trees is None:
            best_trees = self.get_trees(best_delta1)
        
        return best_length ** 2 / self.n, best_trees

print("SimulatedAnnealing class defined")

In [ ]:
# Load baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path)

def load_configuration_from_df(n, existing_df):
    """Load existing configuration from submission CSV."""
    group_data = existing_df[existing_df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"]).replace('s', '')
        y = str(row["y"]).replace('s', '')
        deg = str(row["deg"]).replace('s', '')
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Load all baseline configurations
baseline_configs = {}
baseline_scores = {}
for n in range(1, 201):
    trees = load_configuration_from_df(n, df_baseline)
    baseline_configs[n] = trees
    baseline_scores[n] = calculate_score(trees)

print(f"Loaded baseline configurations for N=1 to N=200")
print(f"Total baseline score: {sum(baseline_scores.values()):.6f}")

In [ ]:
# SA configuration for tessellation
config = {
    "params": {
        "Tmax": 1.0,
        "Tmin": 0.001,
        "nsteps": 500,  # Reduced for faster testing
        "nsteps_per_T": 10,
        "cooling": "exponential",
        "alpha": 0.99,
        "position_delta": 0.02,
        "angle_delta": 2.0,
        "delta1": [0.6, 0.55],  # Initial translation vectors
        "random_state": 42,
        "log_freq": 100,
    }
}

# Initial base trees (blue ~0°, pink ~180°)
initial_trees = [
    ChristmasTree("0", "0", "0"),      # Blue tree
    ChristmasTree("0.3", "0.3", "180")  # Pink tree
]

print("SA configuration ready")

In [ ]:
# Target N values for tessellation with their grid sizes
# nt = [rows, cols] where rows * cols >= N
tessellation_targets = {
    64: [4, 8],   # 4*8=32, need 64 -> try [8, 8]
    72: [4, 9],   # 4*9=36, need 72 -> try [6, 12]
    81: [5, 9],   # 5*9=45, need 81 -> try [9, 9]
    100: [5, 10], # 5*10=50, need 100 -> try [10, 10]
    110: [5, 11], # 5*11=55, need 110 -> try [10, 11]
    121: [6, 11], # 6*11=66, need 121 -> try [11, 11]
    144: [6, 12], # 6*12=72, need 144 -> try [12, 12]
    156: [6, 13], # 6*13=78, need 156 -> try [12, 13]
    169: [7, 13], # 7*13=91, need 169 -> try [13, 13]
    196: [7, 14], # 7*14=98, need 196 -> try [14, 14]
    200: [7, 15], # 7*15=105, need 200 -> try [10, 20]
}

# Correct grid sizes to ensure rows * cols >= N
tessellation_targets = {
    64: [8, 8],
    72: [6, 12],
    81: [9, 9],
    100: [10, 10],
    110: [10, 11],
    121: [11, 11],
    144: [12, 12],
    156: [12, 13],
    169: [13, 13],
    196: [14, 14],
    200: [10, 20],
}

print("Tessellation targets defined:")
for n, nt in tessellation_targets.items():
    print(f"  N={n}: grid {nt[0]}x{nt[1]} = {nt[0]*nt[1]} cells")

In [ ]:
# Test tessellation on N=100 first
test_n = 100
test_nt = tessellation_targets[test_n]

print(f"Testing tessellation for N={test_n} with grid {test_nt[0]}x{test_nt[1]}")
print(f"Baseline score for N={test_n}: {baseline_scores[test_n]:.6f}")

# Reset initial trees
initial_trees = [
    ChristmasTree("0", "0", "0"),
    ChristmasTree("0.3", "0.3", "180")
]

config["params"]["nt"] = test_nt
config["params"]["n"] = test_n
config["params"]["nsteps"] = 300  # Quick test

sa = SimulatedAnnealing(initial_trees, **config["params"])
score, trees_test = sa.solve()

print(f"\nTessellation score for N={test_n}: {score:.6f}")
print(f"Baseline score for N={test_n}: {baseline_scores[test_n]:.6f}")
print(f"Improvement: {baseline_scores[test_n] - score:.6f}")

In [ ]:
# Check for collisions in the result
if has_collision(trees_test):
    print("WARNING: Tessellation result has collisions!")
else:
    print("No collisions in tessellation result")

# Visualize
plt.figure(figsize=(10, 10))
for t in trees_test:
    plt.plot(*t.polygon.exterior.xy)
plt.axis('equal')
plt.title(f'Tessellation N={test_n}, Score={score:.4f}')
plt.savefig('/home/code/experiments/002_tessellation/tessellation_n100.png')
plt.show()

In [ ]:
# Run tessellation for all target N values
tessellation_results = {}

for n, nt in tessellation_targets.items():
    print(f"\n{'='*50}")
    print(f"Running tessellation for N={n} with grid {nt[0]}x{nt[1]}")
    
    # Reset initial trees with different starting angles
    initial_trees = [
        ChristmasTree("0", "0", "0"),
        ChristmasTree("0.3", "0.3", "180")
    ]
    
    config["params"]["nt"] = nt
    config["params"]["n"] = n
    config["params"]["nsteps"] = 500  # More iterations
    config["params"]["random_state"] = 42 + n  # Different seed per N
    
    sa = SimulatedAnnealing(initial_trees, **config["params"])
    score, trees = sa.solve()
    
    # Check for collisions
    if has_collision(trees):
        print(f"WARNING: N={n} has collisions - skipping")
        continue
    
    tessellation_results[n] = {
        'score': score,
        'trees': trees,
        'baseline_score': baseline_scores[n],
        'improvement': baseline_scores[n] - score
    }
    
    print(f"Tessellation score: {score:.6f}")
    print(f"Baseline score: {baseline_scores[n]:.6f}")
    print(f"Improvement: {baseline_scores[n] - score:.6f}")

In [ ]:
# Summary of tessellation results
print("\n" + "="*60)
print("TESSELLATION RESULTS SUMMARY")
print("="*60)

total_improvement = 0
improved_ns = []

for n, result in sorted(tessellation_results.items()):
    imp = result['improvement']
    status = "BETTER" if imp > 0 else "WORSE"
    print(f"N={n}: Tess={result['score']:.6f}, Base={result['baseline_score']:.6f}, Imp={imp:+.6f} [{status}]")
    if imp > 0:
        total_improvement += imp
        improved_ns.append(n)

print(f"\nTotal potential improvement: {total_improvement:.6f}")
print(f"N values improved: {improved_ns}")

In [ ]:
# Create new submission combining baseline with tessellation improvements
def to_str(x):
    return f"s{float(x)}"

# Start with baseline
new_configs = {n: baseline_configs[n] for n in range(1, 201)}

# Replace with tessellation results where they're better
for n, result in tessellation_results.items():
    if result['improvement'] > 0:
        new_configs[n] = result['trees']
        print(f"Replacing N={n} with tessellation (improvement: {result['improvement']:.6f})")

# Calculate new total score
new_total_score = 0
for n in range(1, 201):
    new_total_score += calculate_score(new_configs[n])

print(f"\nNew total score: {new_total_score:.6f}")
print(f"Baseline total score: {sum(baseline_scores.values()):.6f}")
print(f"Total improvement: {sum(baseline_scores.values()) - new_total_score:.6f}")

In [ ]:
# Save submission
rows = []
for n in range(1, 201):
    for i, tree in enumerate(new_configs[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': to_str(tree.center_x),
            'y': to_str(tree.center_y),
            'deg': to_str(tree.angle)
        })

df_submission = pd.DataFrame(rows)
df_submission.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved submission with {len(df_submission)} rows")

# Verify
print(df_submission.head())

In [ ]:
# Save metrics
metrics = {
    'cv_score': new_total_score,
    'baseline_score': sum(baseline_scores.values()),
    'improvement': sum(baseline_scores.values()) - new_total_score,
    'tessellation_results': {n: {'score': r['score'], 'baseline': r['baseline_score'], 'improvement': r['improvement']} 
                             for n, r in tessellation_results.items()}
}

with open('/home/code/experiments/002_tessellation/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"Metrics saved")
print(f"CV Score: {new_total_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap: {new_total_score - 68.894234:.6f}")

In [ ]:
# Final validation - check all N values for overlaps
print("Validating all configurations for overlaps...")
has_issues = False
for n in range(1, 201):
    if has_collision(new_configs[n]):
        print(f"WARNING: N={n} has overlapping trees!")
        has_issues = True

if not has_issues:
    print("All configurations validated - no overlaps detected")